In [11]:
import sys
base = "./../../../"
sys.path.append(base)

In [12]:
import json
import pandas
import pairs_flat_v2 as pairs
import helper
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
from tqdm import tqdm_notebook as tqdm
from pathlib import Path

tqdm.monitor_interval = 0
init_notebook_mode(connected=True)

In [13]:
# Load matrix
gencounts_oscope = pandas.read_csv(Path(base + "data/GSE64016_H1andFUCCI_normalized_EC_human.csv"))

# Set index right
gencounts_oscope.set_index("Unnamed: 0", inplace=True)

# Subset sorted
gencounts_oscope_sorted = gencounts_oscope.iloc[:, 
                                                       [gencounts_oscope.columns.get_loc(c) 
                                                        for c in gencounts_oscope.columns 
                                                        if "G1_" in c or "G2_" in c or "S_" in c]]

# Define annotation
is_G1 = [gencounts_oscope_sorted.columns.get_loc(c) for c in gencounts_oscope_sorted.columns if "G1_" in c]
is_S = [gencounts_oscope_sorted.columns.get_loc(c) for c in gencounts_oscope_sorted.columns if "S_" in c]
is_G2M = [gencounts_oscope_sorted.columns.get_loc(c) for c in gencounts_oscope_sorted.columns if "G2_" in c]

annotation = {
    "G1": list(is_G1),
    "S": list(is_S),
    "G2M": list(is_G2M)
}

go_0007049 = [line.replace("\n","").replace("\r","") for line in open(base + "data/go_0007049_homoSapiens.csv", "r")]
cycle_base = [line.split("\t")[0] for i, line in enumerate(open(base + "data/cyclebase_top1000_genes.tsv", "r")) if 0 < i]
cycle_genes = np.unique(np.concatenate((go_0007049, cycle_base),0))

cc_marker = pairs.sandbag(gencounts_oscope_sorted, phases=annotation, subset_genes=list(cycle_genes), fraction=0.6, processes=10, verbose=True, weighted=True)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 10 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


In [14]:
cc_marker

{'G1': [('AASDHPPT', 'ANKRD17', 0.6842105263157895),
  ('AASDHPPT', 'BIRC6', 0.6923076923076923),
  ('AASDHPPT', 'BRCA2', 0.6882591093117408),
  ('AASDHPPT', 'CDK1', 0.8704453441295547),
  ('AASDHPPT', 'KHDRBS1', 0.680161943319838),
  ('AASDHPPT', 'KIF5B', 0.6639676113360324),
  ('AASDHPPT', 'NCAPG', 0.680161943319838),
  ('AASDHPPT', 'NUP205', 0.6923076923076923),
  ('AASDHPPT', 'POGZ', 0.6842105263157895),
  ('AASDHPPT', 'RBBP8', 0.680161943319838),
  ('AASDHPPT', 'RBX1', 0.6437246963562753),
  ('AASDHPPT', 'SENP5', 0.6558704453441295),
  ('AASDHPPT', 'SPIN1', 0.6153846153846154),
  ('AASDHPPT', 'TCF7L2', 0.7165991902834008),
  ('AASDHPPT', 'WNK1', 0.680161943319838),
  ('ABCF2', 'CDK1', 0.805668016194332),
  ('ABCF2', 'UBE2C', 0.757085020242915),
  ('CCNE2', 'ABL1', 0.659919028340081),
  ('MANF', 'ABL1', 0.6720647773279352),
  ('CDCA7', 'ACACA', 0.680161943319838),
  ('HILPDA', 'ACAP2', 0.6153846153846154),
  ('CDCA7', 'ACSL3', 0.6761133603238867),
  ('ACSL3', 'CDK1', 0.740890688259

In [15]:
gencounts_EMATB6142 = pandas.read_csv(Path(base + "data/E-MTAB-6142_human.csv"), sep=';')
gencounts_EMATB6142.set_index("Gene_ID", inplace=True)
gene_map = {}

with open(base + "data/biomart_human-genes.txt", "r") as f:
    for line in f:
        info = line.split(",")
        gene_map[info[0].replace("\n","").replace("\r","")] = info[1].replace("\n","")

index_list = gencounts_EMATB6142.index.tolist()

for idx, i in enumerate(index_list):
    try:
        if "." in i:
            index_list[idx] = gene_map[i[:i.index(".")]]
        else:
            index_list[idx] = gene_map[i] 
    except KeyError:
        pass

gencounts_EMATB6142.index = index_list
#gencounts_EMATB6142 = gencounts_EMATB6142[~gencounts_EMATB6142.index.duplicated(keep=False)]
gencounts_EMATB6142.head(10)

,S1_G1,S2_G1,S3_G1,S4_G1,S5_G1,S6_G1,S7_G1,S8_G1,S9_G1,S10_G1,...,S87_G2M,S88_G2M,S89_G2M,S90_G2M,S91_G2M,S92_G2M,S93_G2M,S94_G2M,S95_G2M,S96_G2M
TSPAN6,360,5,437,136,328,253,1101,39,157,253,...,391,429,148,397,424,317,403,280,470,725
TNMD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DPM1,111,421,70,179,93,57,60,35,174,91,...,63,228,173,115,87,308,92,179,164,104
SCYL3,2,5,0,0,1,1,3,0,5,1,...,3,0,2,1,2,0,0,0,38,1
C1orf112,179,448,0,0,135,47,0,0,0,159,...,151,68,147,84,151,11,0,0,169,77
FGR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CFH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,34,0,0,0
FUCA2,293,0,0,389,65,229,106,95,112,205,...,175,110,375,46,55,291,283,224,293,93
GCLC,0,0,0,0,208,0,0,0,0,0,...,0,0,0,0,0,0,40,0,0,0
NFYA,0,118,0,0,0,0,0,60,0,0,...,10,0,0,98,108,149,60,125,46,2


In [16]:
x = gencounts_EMATB6142.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_EMATB6142_Qnorm = pandas.DataFrame(X_std.T, index=gencounts_EMATB6142.index, columns=gencounts_EMATB6142.columns)

gencounts_EMATB6142_Qnorm.head(10)

,S1_G1,S2_G1,S3_G1,S4_G1,S5_G1,S6_G1,S7_G1,S8_G1,S9_G1,S10_G1,...,S87_G2M,S88_G2M,S89_G2M,S90_G2M,S91_G2M,S92_G2M,S93_G2M,S94_G2M,S95_G2M,S96_G2M
TSPAN6,3.473684e-01,1.000000e-07,5.264640e-01,3.131861e-02,3.050698e-01,1.526527e-01,9.789371e-01,1.029057e-02,5.297821e-02,1.526527e-01,...,4.210878e-01,4.945908e-01,4.212460e-02,4.369369e-01,4.735360e-01,2.738559e-01,4.525139e-01,2.210190e-01,6.156156e-01,8.947715e-01
TNMD,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
DPM1,3.683954e-01,9.890796e-01,1.736737e-01,6.896897e-01,2.945035e-01,8.958959e-02,1.052632e-01,4.754755e-02,6.416416e-01,2.631579e-01,...,1.157895e-01,8.105044e-01,6.210526e-01,3.895449e-01,2.316756e-01,9.473313e-01,2.787788e-01,6.896897e-01,5.735736e-01,3.313313e-01
SCYL3,7.157157e-01,8.683684e-01,1.000000e-07,1.000000e-07,5.470470e-01,5.470470e-01,8.108108e-01,1.000000e-07,8.683684e-01,5.470470e-01,...,8.108108e-01,1.000000e-07,7.157157e-01,5.470470e-01,7.157157e-01,1.000000e-07,1.000000e-07,1.000000e-07,9.314314e-01,5.470470e-01
C1orf112,9.158360e-01,9.999999e-01,1.000000e-07,1.000000e-07,8.213680e-01,4.843728e-01,1.000000e-07,1.000000e-07,1.000000e-07,8.946219e-01,...,8.683684e-01,5.685140e-01,8.426397e-01,6.526420e-01,8.683684e-01,3.579496e-01,1.000000e-07,1.000000e-07,9.052632e-01,6.314140e-01
FGR,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
CFH,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.894737e-01,1.000000e-07,1.000000e-07,1.000000e-07
FUCA2,9.104104e-01,1.000000e-07,1.000000e-07,9.890842e-01,2.632633e-01,7.893646e-01,4.630937e-01,4.159159e-01,4.894895e-01,7.050920e-01,...,6.315873e-01,4.738216e-01,9.789474e-01,1.683129e-01,2.367367e-01,8.947837e-01,8.738128e-01,7.789999e-01,9.104104e-01,3.948949e-01
GCLC,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.789681e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.367956e-01,1.000000e-07,1.000000e-07,1.000000e-07
NFYA,1.000000e-07,8.368368e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,6.051051e-01,1.000000e-07,1.000000e-07,...,5.051710e-01,1.000000e-07,1.000000e-07,7.266548e-01,7.788708e-01,9.259958e-01,6.051051e-01,8.633743e-01,5.788832e-01,4.579580e-01


In [17]:
EMATB6142_prediction = pairs.cyclone(gencounts_EMATB6142_Qnorm, cc_marker, verbose=True, weighted=True, processes=0)

[__set_matrix] Original Matrix 'x' has shape 59838 x 96
[__set_matrix] Matrix truncation done. Working with 59838 genes for 96 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 412 marker pairs. 8146 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G1: 34, S: 13, G2M: 49


In [18]:
EMATB6142_prediction_table = helper.get_prediction_table(EMATB6142_prediction)
helper.DataTable(EMATB6142_prediction_table)

,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
S1_G1,0.996,0.000,1.000,0.498998,0.000000,0.501002,G1
S2_G1,0.993,0.223,0.000,0.816612,0.183388,0.000000,G1
S3_G1,0.937,0.314,0.000,0.749001,0.250999,0.000000,G1
S4_G1,0.990,0.515,0.000,0.657807,0.342193,0.000000,G1
S5_G1,0.276,0.293,0.988,0.177264,0.188182,0.634554,S
S6_G1,0.653,0.074,0.999,0.378331,0.042874,0.578795,G1
S7_G1,0.951,0.936,0.000,0.503975,0.496025,0.000000,G1
S8_G1,0.979,0.000,0.421,0.699286,0.000000,0.300714,G1
S9_G1,0.999,0.864,0.000,0.536232,0.463768,0.000000,G1


In [19]:
EMATB6142_labels = list(['G1'] * 32) + list(['S'] * 32) + list(['G2M'] * 32)
EMATB6142_evaluation = helper.evaluate_prediction(EMATB6142_prediction_table, EMATB6142_labels)
helper.plot_evaluation(*EMATB6142_evaluation, xaxis=["G1","S","G2M"], xaxislbl="Phase", average=True)

F1 Score: G1: 0.8484848484848485, S: 0.4888888888888889, G2M: 0.6666666666666666
Reacall: G1: 0.875, S: 0.34375, G2M: 0.84375 
Precision: G1: 0.8235294117647058, S: 0.8461538461538461, G2M: 0.5510204081632653 


{'data': [{'marker': {'color': 'red', 'size': 10, 'symbol': 'circle'},
   'mode': 'markers',
   'name': 'F1-Score',
   'type': 'scatter',
   'x': ['G1', 'S', 'G2M'],
   'y': array([0.84848485, 0.48888889, 0.66666667])},
  {'marker': {'color': 'blue', 'size': 10, 'symbol': 'square'},
   'mode': 'markers',
   'name': 'Recall-Score',
   'type': 'scatter',
   'x': ['G1', 'S', 'G2M'],
   'y': array([0.875  , 0.34375, 0.84375])},
  {'marker': {'color': 'green', 'size': 10, 'symbol': 'triangle-up'},
   'mode': 'markers',
   'name': 'Precision-Score',
   'type': 'scatter',
   'x': ['G1', 'S', 'G2M'],
   'y': array([0.82352941, 0.84615385, 0.55102041])},
  {'marker': {'color': 'red', 'size': 10},
   'mode': 'lines',
   'name': 'Average F1',
   'type': 'scatter',
   'x': ['G1', 'S', 'G2M'],
   'y': [0.6680134680134681, 0.6680134680134681, 0.6680134680134681]},
  {'marker': {'color': 'blue', 'size': 10},
   'mode': 'lines',
   'name': 'Average Recall',
   'type': 'scatter',
   'x': ['G1', 'S', 'G